<a href="https://colab.research.google.com/github/chongzicbo/nlp-ml-dl-notes/blob/master/TextCNN%E8%AF%A6%E8%A7%A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#(一)TextCNN中的卷积操作
卷积神经网络的核心思想是捕捉局部特征，对于文本来说，局部特征就是由若干单词组成的滑动窗口，类似于N-gram。卷积神经网络的优势在于能够自动地对N-gram特征进行组合和筛选，获得不同抽象层次的语义信息。

图1是论文[1]中用于文本分类的卷积神经网络模型架构。

<img src="https://pic2.zhimg.com/80/v2-38e6e46009ea88c06465ed0770051c4d_720w.jpg">
<center>图1 双通道的TextCNN结构<center/>

<img src="https://pic3.zhimg.com/80/v2-2ea1f0b8b166f31273b26bca3ba8e8b2_720w.jpg">

<center>图2 用于文本分类任务的TextCNN结构描述 这里详细解释TextCNN架构及词向量矩阵是如何做卷积的。<center/>



第(2)部分是重点，理解好卷积过程是关键。

（1）第一层为输入层。输入层是一个$n \times k$ 的矩阵，其中 $n$ 为一个句子中的单词数， $k$ 是每个词对应的词向量的维度。也就是说，输入层的每一行就是一个单词所对应的$k$维的词向量。另外，这里为了使向量长度一致对原句子进行了padding操作。我们这里使用 $x_i \in R^k$表示句子中第 $i$个单词的 $k$维词嵌入。

每个词向量可以是预先在其他语料库中训练好的，也可以作为未知的参数由网络训练得到。这两种方法各有优势，预先训练的词嵌入可以利用其他语料库得到更多的先验知识，而由当前网络训练的词向量能够更好地抓住与当前任务相关联的特征。因此，图中的输入层实际采用了双通道的形式，即有两个 $n \times k$ 的输入矩阵，其中一个用预训练好的词嵌入表达，并且在训练过程中不再发生变化；另外一个也由同样的方式初始化，但是会作为参数，随着网络的训练过程发生改变。

（2）第二层为卷积层，第三层为池化层。

首先，我们要注意到卷积操作在计算机视觉(CV)和NLP中的不同之处。在CV中，卷积核往往都是正方形的，比如 $3 \times 3$ 的卷积核，然后卷积核在整张image上沿高和宽按步长移动进行卷积操作。与CV中不同的是，在NLP中输入层的"image"是一个由词向量拼成的词矩阵，且卷积核的宽和该词矩阵的宽相同，该宽度即为词向量大小，且卷积核只会在高度方向移动。因此，每次卷积核滑动过的位置都是完整的单词，不会将几个单词的一部分"vector"进行卷积，词矩阵的行表示离散的符号（也就是单词）[2]，这就保证了word作为语言中最小粒度的合理性（当然，如果研究的粒度是character-level而不是word-level，需要另外的方式处理）。

然后，我们详述这个卷积、池化过程。由于卷积核和word embedding的宽度一致，一个卷积核对于一个sentence，卷积后得到的结果是一个vector，其shape=(sentence_len - filter_window_size + 1, 1)，那么，在经过max-pooling操作后得到的就是一个Scalar。我们会使用多个filter_window_size（原因是，这样不同的kernel可以获取不同范围内词的关系，获得的是纵向的差异信息，即类似于n-gram，也就是在一个句子中不同范围的词出现会带来什么信息。比如可以使用3,4,5个词数分别作为卷积核的大小），每个filter_window_size又有num_filters个卷积核（原因是卷积神经网络学习的是卷积核中的参数，每个filter都有自己的关注点，这样多个卷积核就能学习到多个不同的信息。[2]中也提到使用多个相同size的filter是为了从同一个窗口学习相互之间互补的特征。比如可以设置size为3的filter有64个卷积核）。一个卷积核经过卷积操作只能得到一个scalar，将相同filter_window_size卷积出来的num_filter个scalar组合在一起，组成这个filter_window_size下的feature_vector。最后再将所有filter_window_size下的feature_vector也组合成一个single vector，作为最后一层softmax的输入。对这个过程若有不清楚的地方，可以对照着图2(该图来自论文[2])来看，图2非常完美地诠释了这个过程。


最后是池化层。如图中所示的网络采用了1-Max池化，即为从每个滑动窗口产生的特征向量中筛选出一个最大的特征，然后将这些特征拼接起来构成向量表示。也可以选用K-Max池化（选出每个特征向量中最大的K个特征），或者平均池化（将特征向量中的每一维取平均）等，达到的效果都是将不同长度的句子通过池化得到一个定长的向量表示。

在整个过程中，通过训练得到的参数包括：filter们的权值矩阵 $w$们，激活函数中的偏置项 $b$ ，softmax函数中的权重矩阵，若词向量也加入训练进来的话，则包括该word embeddings。

（3）得到文本句子的向量表示之后，后面的网络结构就和具体的任务相关了。本例中展示的是一个文本分类的场景，因此最后接入了一个全连接层，并使用Softmax激活函数输出每个类别的概率。

(二)TextCNN的超参数调参
首先，我们默认的TextCNN模型超参数一般都是这种配置。如下表：


这里将一下调参的问题，主要方法来自论文[2]。在最简单的仅一层卷积的TextCNN结构中，下面的超参数都对模型表现有影响：
<img src="https://pic1.zhimg.com/80/v2-a0de86fee7c073e95ee325fea3ba21f8_720w.jpg">

输入词向量表征：词向量表征的选取(如选word2vec还是GloVe)
卷积核大小：一个合理的值范围在1~10。若语料中的句子较长，可以考虑使用更大的卷积核。另外，可以在寻找到了最佳的单个filter的大小后，尝试在该filter的尺寸值附近寻找其他合适值来进行组合。实践证明这样的组合效果往往比单个最佳filter表现更出色
feature map特征图个数：主要考虑的是当增加特征图个数时，训练时间也会加长，因此需要权衡好。当特征图数量增加到将性能降低时，可以加强正则化效果，如将dropout率提高过0.5
激活函数：ReLU和tanh是最佳候选者
池化策略：1-max pooling表现最佳
正则化项(dropout/L2)：相对于其他超参数来说，影响较小点